In [ ]:
import keras
import numpy as np
from keras.datasets import mnist

In [ ]:
batchsize = 32


In [ ]:
def encoder():
    # Q(z|X)
    # l1 = Dense(512, activation = 'relu')
    # L2 = Dense(n_z, activation = 'linear') where n_z = number of samples
    # Output: mu, log_sigma 

In [ ]:
def decoder(): 
    # P(X|z)
    # hidden_layer: Dense(512, activation='relu')
    # output_layer: Dense(784, activation='sigmoid')
    # outputs = output_layer(hidden_layer(x))

In [ ]:
def loss():
    # p_z = gaussian(0,1)
    # q_z_x = gaussian(mean_y, e^ln_var_y) - check this exp
    # p_x_z = gaissian(mean_x, e^ln_var_x)
    # L = F.gaussian_kl_dvergance(q_z_x, p_z) - F.gaussian_nll(p_x_z)
        
    #? What about input x?
        
    # Why use log variance instead of variance:

In [ ]:
def sample_z(args):
    mu, log_sigma = args
    eps = K.random_normal(shape=(m, n_z), mean=0., std=1.)
    return mu + K.exp(log_sigma / 2) * eps

In [ ]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
batches = int(len(x_train)/batchsize)

In [ ]:
training_set = img_load(data_file[:int(.64 * len(data_file))])
validation_set = img_load(data_file[int(.64 * len(data_file)) : int(.8 * len(data_file))])
test_set = img_load(data_file[int(.8 * len(data_file) ):])

batches = int(len(training_set)/BATCH_SIZE)

In [ ]:
def batch_generator():
    
    while True:

        for batch in range(batches):
            
            data = x_train[batch * batchsize: (batch + 1) * batchsize]
            
            # the data is also the target (since we're re-generating images)
            yield (data, data)

In [ ]:
model.fit_generator(batch_generator(), steps_per_epoch = batches, epochs=10, shuffle = True, callbacks = callback_list)